In [1]:
import sys; sys.path.append('..') # add project root to the python path

In [2]:
import gym
import torch

from src.part3.MLP import MultiLayerPerceptron as MLP
from src.part4.PolicyGradient import REINFORCE
from src.common.train_utils import EMAMeter, to_tensor

In [3]:
env = gym.make('CartPole-v1')
s_dim = env.observation_space.shape[0]
a_dim = env.action_space.n

C:\Users\joon0\Anaconda3\envs\gpu_torch130\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


## REINFORCE 알고리즘으로, 'CartPole-V1'을 정복하기.

이번 실습에서는 `REINFORCE` 알고리즘으로 `CartPole-V1` MDP를 정복해볼까요?
`REINFORCE`의 의사코드는 다음과 같습니다.

<img src="./images/REINFORCE.png" width="60%" height="40%" title="REINFORCE" alt="REINFORCE"></img>

`REINFORCE` 알고리즘을 파이썬으로는 어떻게 구현할까요?
```python
import torch.nn as nn
from torch.distributions.categorical import Categorical

class REINFORCE(nn.Module):

    def __init__(self,
                 policy: nn.Module,
                 gamma: float = 1.0,
                 lr: float = 0.0002):
        super(REINFORCE, self).__init__()
        self.policy = policy  # make sure that 'policy' returns logits!
        self.gamma = gamma
        self.opt = torch.optim.Adam(params=self.policy.parameters(),
                                    lr=lr)

        self._eps = 1e-25

    def get_action(self, state):
        with torch.no_grad():
            logits = self.policy(state)
            dist = Categorical(logits=logits)
            a = dist.sample()  # sample action from softmax policy
        return a
```

### Logit? Catergorical?

Catergorical distribution (다항분포) 는 이항분포의 확장판입니다. <br>

>이항분포는 사건의 종류가 2개일 상황을 (예를 들어 한번 동전을 던져서 앞/뒤가 나올 확률) 모델링 할 때 자주 쓰이게 됩니다.
이항 분포의 매개변수는 $p$ 로 하나의 사건이 나올 확률을 표현합니다. 이때 $0 \leq p \leq 1$ 를 따르게 됩니다.

그렇다면 사건의 종류가 2개보다 많은 경우는 어떻게 할 수 있을까요? 그런 경우에 다항 분포를 사용해볼 수 있습니다.

> 사건의 종류가 $n$ 개 일때, 다항분포의 매개변수들 $(p_1, ..., p_n)$ 이 되며, $p_i$ 는 각 사건이 일어날 확률을 표현합니다.
> 따라서 $p_i$ 는 모두 0보다 크거나 같고, $\sum_{i=1}^{n} p_i =1$ 을 만족시켜야 합니다.

자 그러면, 이제 신경망 `self.policy(state)` 출력이 위의 조건을 만족시키게 만들 수 있을까요? 다양한 방법이 존재하지만, 가장 일반적인 선택은
softmax라는 연산자를 활용하는 것입니다.

$$\sigma(z)_i = \frac{e^{z_i}}{\sum_{j=1}^{n}e^{z_j}} \forall i=1,...,n$$

임의의 값들의 집합 $\{{z_i}\}_{i=1}^{n}$ 에 위에서 정의한 softmax 연산자를 가해주면, 모든 값들은 0 이상의 값으로 바뀌고 값들의 합은 1.0 이 되게 됩니다.
이때, $z_i$ 를 일반적으로 logit 이라고 부릅니다.

### Implemenation tip! 
> 일반적으로 NN계산 중에 `exp()` 연산은 선호되지 않습니다. 입력값이 작거나/클때 쉽게 값이 underflow/overflow 할 염려가 있기 때문인데요.
> 신경써서 구현하면 그런 문제들을 완화할 수 있지만, 매번 신경쓰기가 번거롭습니다. <br>

> softmax + Catergorical 사용같이 자주 사용되는 패턴은 pytorch 수준에서 최대한 numerical stability를 가지도록 만들어져 있습니다. 그 예시 중에 하나로 pytorch 의 `Catergorical`은 logit값으로 부터 생성하는것이 가능합니다. 따라서, 직접 softmax를 계산한 후 `Catergorical` 매개변수로 사용하는것 보다는 `Catergorical`분포의 매개변수로 `logit`을 넘겨주는 것이 조금 더 안전합니다.


```python
    @staticmethod
    def _pre_process_inputs(episode):
        states, actions, rewards = episode

        # assume inputs as follows
        # s : torch.tensor [num.steps x state_dim]
        # a : torch.tensor [num.steps]
        # r : torch.tensor [num.steps]

        # reversing inputs
        states = states.flip(dims=[0])
        actions = actions.flip(dims=[0])
        rewards = rewards.flip(dims=[0])
        return states, actions, rewards

    def update(self, episode):
        # sample-by-sample update version of REINFORCE
        # sample-by-sample update version is highly inefficient in computation
        states, actions, rewards = self._pre_process_inputs(episode)

        g = 0
        for s, a, r in zip(states, actions, rewards):
            g = r + self.gamma * g
            dist = Categorical(logits=self.policy(s))
            prob = dist.probs[a]

            # Don't forget to put '-' in the front of pg_loss !!!!!!!!!!!!!!!!
            # the default behavior of pytorch's optimizer is to minimize the targets
            # add 'self_eps' to prevent numerical problems of logarithms
            pg_loss = - torch.log(prob + self._eps) * g

            self.opt.zero_grad()

            pg_loss.backward()
            self.opt.step()
```

In [4]:
net = MLP(s_dim, a_dim, [128])
agent = REINFORCE(net)
ema = EMAMeter()

In [5]:
n_eps = 10000
print_every = 500

for ep in range(n_eps):
    s = env.reset()
    cum_r = 0

    states = []
    actions = []
    rewards = []

    while True:
        s = to_tensor(s, size=(1, 4))
        a = agent.get_action(s)
        ns, r, done, info = env.step(a.item())

        states.append(s)
        actions.append(a)
        rewards.append(r)

        s = ns
        cum_r += r
        if done:
            break

    ema.update(cum_r)
    if ep % print_every == 0:
        print("Episode {} || EMA: {} ".format(ep, ema.s))

    states = torch.cat(states, dim=0)  # torch.tensor [num. steps x state dim]
    actions = torch.stack(actions).squeeze()  # torch.tensor [num. steps]
    rewards = torch.tensor(rewards)  # torch.tensor [num. steps]

    episode = (states, actions, rewards)
    agent.update_episode(episode, use_norm=True)

Episode 0 || EMA: 15.0 
Episode 500 || EMA: 28.274833570317945 
Episode 1000 || EMA: 55.207220467799196 
Episode 1500 || EMA: 60.38703727608534 
Episode 2000 || EMA: 115.56517011414884 
Episode 2500 || EMA: 185.65287882975048 
Episode 3000 || EMA: 161.19202441967337 
Episode 3500 || EMA: 254.1207580027799 
Episode 4000 || EMA: 341.3443486106662 
Episode 4500 || EMA: 445.50876492812426 
Episode 5000 || EMA: 316.0545563651241 
Episode 5500 || EMA: 497.89040434755145 
Episode 6000 || EMA: 427.0517173284881 
Episode 6500 || EMA: 468.69471805294796 
Episode 7000 || EMA: 499.99999999637424 
Episode 7500 || EMA: 499.01531979523133 
Episode 8000 || EMA: 283.0781249999658 
Episode 8500 || EMA: 499.9999978169799 
Episode 9000 || EMA: 499.96826171875 
Episode 9500 || EMA: 499.99826049804676 
